In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
class NN(nn.Module):
  def __init__(self,input_size, num_classes):
    super(NN, self).__init__()

    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

model = NN(784, 10)
x = torch.randn(64, 784)

print(model(x).shape)

torch.Size([64, 10])


In [ ]:
print(torch.cuda.get_device_name())

Tesla T4


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print(device)

cuda


In [ ]:
input_size = 784
num_classes=10
learning_rate=0.001
batch_size = 64
num_epochs=1

In [ ]:
train_dataset = datasets.MNIST(train=True, download=True, transform=transforms.ToTensor(), root='dataset/MNIST')
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST(train=False, download=True, transform=transforms.ToTensor(), root='dataset/MNIST')
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

Extracting dataset/MNIST/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/MNIST/raw


Extracting dataset/MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/MNIST/raw


Extracting dataset/MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/MNIST/raw


Extracting dataset/MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/MNIST/raw
Processing...



Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# Initiate the network
model = NN(input_size, num_classes).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)
    #print(data.shape)
    data = data.reshape(data.shape[0], -1)
    #print(data.shape)

    # Forward
    scores = model(data)
    loss = criterion(scores ,targets)

    # Backward
    optimizer.zero_grad()
    loss.backward()

    # Gradient descent
    optimizer.step()

def check_accuracy(loader, model):
  if loader.dataset.train:
    print('Checking accuracy on training data')
  else:
    print('Checking accuracy on test data')
  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)

    print(f'Got {num_correct}/{num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

  model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on training data
Got 58697/60000 with accuracy 97.83
Checking accuracy on test data
Got 9685/10000 with accuracy 96.85
